# Part 0: Dataloader and Visualizations

In [1]:
import torch
import wandb
import scipy.io

import numpy as np

import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

from voc_dataset import VOCDataset

from PIL import Image

from utils import *

USE_WANDB = True

## Editing the Dataloader
The first part of the assignment involves editing the dataloader so that we can access bounding-box proposals as well as the ground-truth bounding boxes. The ground truth bounding box can be accessed through the VOC Dataset annotations itself. Unsupervised bounding box proposals are obtained through methods such as [Selective Search](https://ivi.fnwi.uva.nl/isis/publications/2013/UijlingsIJCV2013/UijlingsIJCV2013.pdf).

Since Selective Search is slow to run on each image, we have pre-computed the bounding box proposals. You should be able to access the `.mat` files using `scipy.io.loadmat('file.mat')`. Feel free to experiment with the data in the files to figure out the number of proposals per image, their scores, etc.

Your task is to change the dataloader to obtain the ground-truth bounding boxes, as well as the proposed bounding boxes for each image. Returning a dictionary would be convenient here. For the bounding boxes, using the relative positions is usually a better idea since they are invariant to changes in the size of the image.

In [2]:
# Load the Dataset - items at a particular index can be accesed by usual indexing notation (dataset[idx])
dataset = VOCDataset('trainval', top_n=10)

In [6]:
#TODO: get the image information from index 2020
idx = 2020

input = dataset[idx]
image = input['image']
gt_classes, gt_boxes, rois = input['gt_classes'], input['gt_boxes'], input['rois']
print("Class of image with index 2020:", VOCDataset.get_class_name(gt_classes[0]))

Class of image with index 2020: train


## Wandb Init and Logging
Initialize a Weights and Biases project, and convert the image tensor to a PIL image and plot it (check `utils.py` for helper functions).

You can use [this](https://docs.wandb.ai/library/log) as a reference for logging syntax.

In [7]:
if USE_WANDB:
    wandb.init(project="vlr2", reinit=True)

wandb: Currently logged in as: jiaqigen (use `wandb login --relogin` to force relogin)
/home/jiaqi/anaconda3/envs/16824_hw2/lib/python3.9/site-packages/IPython/html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "
wandb: wandb version 0.10.23 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


See this block as an example of plotting the ground truth box for an image.

In [9]:
class_id_to_label = dict(enumerate(dataset.CLASS_NAMES))
img = wandb.Image(image, boxes={
    "predictions": {
        "box_data": get_box_data(gt_classes, gt_boxes),
        "class_labels": class_id_to_label,
    },
})
wandb.log({"images": img})

Check the `get_box_data` function in `utils.py` and understand how it is being used. Log the image with the GT bounding box on wandb.
After, this you should be able to easily plot the top 10 bounding proposals as well.

In [14]:
#TODO: plot top ten proposals (of bounding boxes)
nums = range(len(rois)) # placeholder for names of proposals
img = wandb.Image(image, boxes={
    "predictions": {
        "box_data": get_box_data(nums, rois),
        "class_labels": class_id_to_label,
    },
})
wandb.log({"images": img})